In [20]:
import findspark
findspark.init()
import pyspark

from pyspark import SparkConf, SparkContext

from pyspark.sql import SQLContext
from pyspark.sql import SparkSession 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

from sklearn.linear_model import LinearRegression
import time
%matplotlib inline



In [13]:
import time

In [15]:
sc.stop()
from pyspark import SparkConf, SparkContext
conf=(SparkConf()
      .set("spark.cores.max", "15")
      .set("spael.dynamicAllocation.initialExecuters","3")
      .set("spark.executor.cores","5")
      .set("spark.exector.memory","6g"))
sc=pyspark.SparkContext(appName="Pi2",conf=conf)
spark=SparkSession \
    .builder \
    .appName("Python Spark SQL basic example ") \
    .getOrCreate()


In [16]:
sc.getConf().getAll()

[(u'spael.dynamicAllocation.initialExecuters', u'3'),
 (u'spark.cores.max', u'15'),
 (u'spark.rdd.compress', u'True'),
 (u'spark.app.name', u'Python Spark SQL basic example '),
 (u'spark.executor.cores', u'5'),
 (u'spark.driver.port', u'33317'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.master', u'local[*]'),
 (u'spark.executor.id', u'driver'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.driver.host', u'192.168.137.97'),
 (u'spark.exector.memory', u'6g'),
 (u'spark.app.id', u'local-1576493629620')]

In [25]:
df=spark.read.option('header','true')\
        .option('inferSchema','true')\
        .csv('train.csv')
train = pd.read_csv('train.csv')

df_test=spark.read.option('header','true')\
        .option('inferSchema','true')\
        .csv('test.csv')

In [18]:
df_test.cache()

DataFrame[User_ID: int, Product_ID: string, Gender: int, Age: int, Occupation: int, City_Category: string, Stay_In_Current_City_Years: int, Marital_Status: int, Product_Category_1: int, Product_Category_2: int, Product_Category_3: int, City_Category1: int]

In [7]:
df.show()

+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+--------------+
|User_ID|Product_ID|Gender|Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|City_Category1|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+--------------+
|1000001| P00069042|     0| 14|        10|            A|                         2|             0|                 3|              null|              null|    8370|             1|
|1000001| P00248942|     0| 14|        10|            A|                         2|             0|                 1|                 6|                14|   15200|             1|
|1000001| P00087842|     0| 14|        10|            A|                         2|             0|  

In [8]:
df.orderBy('Purchase',ascending= False).limit(10).toPandas()[['Age','Occupation','City_Category1','Stay_In_Current_City_Years','Marital_Status','Purchase']]

,Age,Occupation,City_Category1,Stay_In_Current_City_Years,Marital_Status,Purchase
0,30,0,3,1,0,23961
1,30,17,3,3,0,23961
2,30,4,1,2,1,23961
3,60,0,3,1,1,23960
4,48,1,2,2,1,23960
5,53,20,1,0,1,23960
6,40,12,2,1,0,23960
7,30,0,3,3,0,23959
8,53,13,2,1,1,23959
9,22,4,2,4,0,23958


## Visualization

In [10]:
#sc.stop()

In [11]:

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

sqlContext.registerDataFrameAsTable(df, 'df')

sqlContext.registerDataFrameAsTable(df_test, 'df_test')


cust_py_test = sqlContext.sql('''SELECT * 
                            FROM df_test 
                              ''')

cust_py = sqlContext.sql('''SELECT Gender,City_Category1,Age,Occupation,Stay_In_Current_City_Years,Marital_Status,
                             Purchase    
                            FROM df GROUP BY Gender,Age,City_Category1,Occupation,Stay_In_Current_City_Years,Marital_Status,
                             Purchase
                            ORDER BY Purchase''')



In [26]:
#group by 1
start=time.time()

gen=train.groupby('Gender')['Purchase'].sum().reset_index()

end=time.time()

print("time taken is",end-start)
gen

('time taken is', 0.0956120491027832)


,Gender,Purchase
0,0,1186232642
1,1,3909580100


In [13]:
sqlContext.registerDataFrameAsTable(df,'df')
start=time.time()

v1=sqlContext.sql('''SELECT Gender,sum(Purchase) as Purchase
                            FROM df 
                     GROUP BY Gender
                              ''')
v1.show()
end=time.time()

print(end-start)

+------+----------+
|Gender|  Purchase|
+------+----------+
|     1|3909580100|
|     0|1186232642|
+------+----------+

1.92873191833


DataFrame[User_ID: int, Product_ID: string, Gender: int, Age: int, Occupation: int, City_Category: string, Stay_In_Current_City_Years: int, Marital_Status: int, Product_Category_1: int, Product_Category_2: int, Product_Category_3: int, Purchase: int, City_Category1: int]

In [27]:
#groub by 2
start=time.time()

a=train.groupby(['Gender'])['User_ID'].nunique().reset_index() 

end=time.time()

print("time taken is",end-start)
a

('time taken is', 0.08516383171081543)


,Gender,User_ID
0,0,1666
1,1,4225


In [20]:
sqlContext.registerDataFrameAsTable(df,'df')
start=time.time()

v2=sqlContext.sql('''SELECT Gender,count(Distinct User_ID) 
                            FROM df 
                     GROUP BY Gender
                              ''')
v2.show()
end=time.time()

print(end-start)

+------+-----------------------+
|Gender|count(DISTINCT User_ID)|
+------+-----------------------+
|     1|                   4225|
|     0|                   1666|
+------+-----------------------+

1.44279980659


In [28]:
#groupby3

start=time.time()

age=train.groupby('Age')['Purchase'].sum().reset_index()




end=time.time()

print("time taken is",end-start)
age

('time taken is', 0.01680612564086914)


,Age,Purchase
0,14,134913183
1,22,913848675
2,30,2031770578
3,40,1026569884
4,48,420843403
5,53,367099644
6,60,200767375


In [21]:
sqlContext.registerDataFrameAsTable(df,'df')
start=time.time()

v3=sqlContext.sql('''SELECT Age,sum(Purchase) as Purchase
                            FROM df 
                     GROUP BY Age
                              ''')
v3.show()
end=time.time()

print(end-start)

+---+----------+
|Age|  Purchase|
+---+----------+
| 53| 367099644|
| 22| 913848675|
| 40|1026569884|
| 48| 420843403|
| 60| 200767375|
| 14| 134913183|
| 30|2031770578|
+---+----------+

0.699874162674


In [29]:
#group by 4

start=time.time()

occ=train.groupby('Occupation')['Purchase'].sum().reset_index()


end=time.time()

print("time taken is",end-start)
occ

('time taken is', 0.0203859806060791)


,Occupation,Purchase
0,0,635406958
1,1,424614144
2,2,238028583
3,3,162002168
4,4,666244484
5,5,113649759
6,6,188416784
7,7,557371587
8,8,14737388
9,9,54340046


In [22]:
sqlContext.registerDataFrameAsTable(df,'df')
start=time.time()

v4=sqlContext.sql('''SELECT Occupation,sum(Purchase) as Purchase
                            FROM df 
                     GROUP BY Occupation
                              ''')
v4.show()
end=time.time()

print(end-start)

+----------+---------+
|Occupation| Purchase|
+----------+---------+
|        12|305449446|
|         1|424614144|
|        13| 71919481|
|        16|238346955|
|         6|188416784|
|         3|162002168|
|        20|296570442|
|         5|113649759|
|        19| 73700617|
|        15|118960211|
|         9| 54340046|
|        17|393281453|
|         4|666244484|
|         8| 14737388|
|         7|557371587|
|        10|115844465|
|        11|106751618|
|        14|259454692|
|         2|238028583|
|         0|635406958|
+----------+---------+
only showing top 20 rows

1.47269701958


In [30]:
#groupby5
start=time.time()

cit=train.groupby('City_Category')['Purchase'].sum().reset_index()


end=time.time()

print("time taken is",end-start)
cit


('time taken is', 0.027085065841674805)


,City_Category,Purchase
0,A,1316471661
1,B,2115533605
2,C,1663807476


In [23]:
sqlContext.registerDataFrameAsTable(df,'df')
start=time.time()

v5=sqlContext.sql('''SELECT City_Category,sum(Purchase) as Purchase
                            FROM df 
                     GROUP BY City_Category
                              ''')
v5.show()
end=time.time()

print(end-start)

+-------------+----------+
|City_Category|  Purchase|
+-------------+----------+
|            B|2115533605|
|            C|1663807476|
|            A|1316471661|
+-------------+----------+

1.04278111458


In [31]:
#groupby6

start=time.time()

stay=train.groupby('Stay_In_Current_City_Years')['Purchase'].sum().reset_index()

end=time.time()

print("time taken is",end-start)
stay

('time taken is', 0.01528310775756836)


,Stay_In_Current_City_Years,Purchase
0,0,682979229
1,1,1792872533
2,2,949173931
3,3,884902659
4,4,785884390


In [24]:
sqlContext.registerDataFrameAsTable(df,'df')
start=time.time()

v6=sqlContext.sql('''SELECT Stay_In_Current_City_Years,sum(Purchase) as Purchase
                            FROM df 
                     GROUP BY Stay_In_Current_City_Years
                              ''')
v6.show()
end=time.time()

print(end-start)

+--------------------------+----------+
|Stay_In_Current_City_Years|  Purchase|
+--------------------------+----------+
|                         1|1792872533|
|                         3| 884902659|
|                         4| 785884390|
|                         2| 949173931|
|                         0| 682979229|
+--------------------------+----------+

0.506909132004


In [32]:
#group by 7


start=time.time()

cs=train.groupby(['City_Category','Stay_In_Current_City_Years'])['Purchase'].sum().reset_index()  

end=time.time()

print("time taken is",end-start)
cs

('time taken is', 0.061424970626831055)


,City_Category,Stay_In_Current_City_Years,Purchase
0,A,0,217277717
1,A,1,437761817
2,A,2,243224188
3,A,3,221128667
4,A,4,197079272
5,B,0,255389454
6,B,1,765647432
7,B,2,384010246
8,B,3,392329940
9,B,4,318156533


In [25]:
sqlContext.registerDataFrameAsTable(df,'df')
start=time.time()
v7=sqlContext.sql('''SELECT City_Category,Stay_In_Current_City_Years,sum(Purchase) as Purchase
                            FROM df 
                     GROUP BY City_Category,Stay_In_Current_City_Years
                              ''')
v7.show()
end=time.time()

print(end-start)

+-------------+--------------------------+---------+
|City_Category|Stay_In_Current_City_Years| Purchase|
+-------------+--------------------------+---------+
|            C|                         3|271444052|
|            B|                         0|255389454|
|            A|                         3|221128667|
|            A|                         0|217277717|
|            A|                         2|243224188|
|            A|                         1|437761817|
|            C|                         1|589463284|
|            B|                         1|765647432|
|            B|                         3|392329940|
|            C|                         2|321939497|
|            A|                         4|197079272|
|            B|                         4|318156533|
|            C|                         0|210312058|
|            C|                         4|270648585|
|            B|                         2|384010246|
+-------------+--------------------------+----

In [33]:
#groub by 8

start=time.time()

mar=train.groupby('Marital_Status')['Purchase'].sum().reset_index()

end=time.time()

print("time taken is",end-start)
mar

('time taken is', 0.016730070114135742)


,Marital_Status,Purchase
0,0,3008927447
1,1,2086885295


In [26]:
sqlContext.registerDataFrameAsTable(df,'df')
start=time.time()

v8=sqlContext.sql('''SELECT Marital_Status,sum(Purchase) as Purchase
                            FROM df 
                     GROUP BY Marital_Status
                              ''')
v8.show()
end=time.time()

print(end-start)

+--------------+----------+
|Marital_Status|  Purchase|
+--------------+----------+
|             1|2086885295|
|             0|3008927447|
+--------------+----------+

0.792418003082


In [34]:
#groupby 9

start=time.time()

a=train.groupby(['Gender','Marital_Status'])['User_ID'].nunique().reset_index() 

end=time.time()

print("time taken is",end-start)
a

('time taken is', 0.11162519454956055)


,Gender,Marital_Status,User_ID
0,0,0,947
1,0,1,719
2,1,0,2470
3,1,1,1755


In [27]:
sqlContext.registerDataFrameAsTable(df,'df')
start=time.time()

v9=sqlContext.sql('''SELECT Marital_Status,Gender,count(Distinct User_ID) 
                            FROM df 
                     GROUP BY Marital_Status,Gender
                              ''')
v9.show()
end=time.time()

print(end-start)

+--------------+------+-----------------------+
|Marital_Status|Gender|count(DISTINCT User_ID)|
+--------------+------+-----------------------+
|             1|     0|                    719|
|             1|     1|                   1755|
|             0|     0|                    947|
|             0|     1|                   2470|
+--------------+------+-----------------------+

23.0301530361


In [35]:

#groupby 10
start=time.time()

pop = train.groupby(by=['City_Category',"Gender",'Marital_Status'], as_index=False).agg({'User_ID': pd.Series.nunique,'Purchase':'sum'})

end=time.time()

print("time taken is",end-start)

pop

('time taken is', 0.11485004425048828)


,City_Category,Gender,Marital_Status,Purchase,User_ID
0,A,0,0,187268915,189
1,A,0,1,119061000,106
2,A,1,0,631081711,463
3,A,1,1,379060035,287
4,B,0,0,292807837,288
5,B,0,1,200809171,215
6,B,1,0,957797651,716
7,B,1,1,664118946,488
8,C,0,0,204077375,470
9,C,0,1,182208344,398


In [28]:
sqlContext.registerDataFrameAsTable(df,'df')
start=time.time()
v10=sqlContext.sql('''SELECT City_Category,Gender,Marital_Status,count(User_ID) as User_ID, sum(Purchase) as Purchase
                            FROM df 
                     GROUP BY City_Category,Gender,Marital_Status
                     order by City_Category,gender
                              ''')
v10.show()
end=time.time()

print(end-start)

+-------------+------+--------------+-------+---------+
|City_Category|Gender|Marital_Status|User_ID| Purchase|
+-------------+------+--------------+-------+---------+
|            A|     0|             0|  21827|187268915|
|            A|     0|             1|  13877|119061000|
|            A|     1|             0|  69346|631081711|
|            A|     1|             1|  42670|379060035|
|            B|     0|             0|  34527|292807837|
|            B|     0|             1|  23269|200809171|
|            B|     1|             1|  71183|664118946|
|            B|     1|             0| 102194|957797651|
|            C|     0|             1|  19842|182208344|
|            C|     0|             0|  22467|204077375|
|            C|     1|             0|  74370|735893958|
|            C|     1|             1|  54496|541627799|
+-------------+------+--------------+-------+---------+

5.97042322159


In [36]:
#groub by 11
start=time.time()

prod=train.groupby(['Product_Category_1'])['Purchase'].sum().reset_index().sort_values(['Purchase'],ascending=False)

end=time.time()

print("time taken is",end-start)
prod

('time taken is', 0.0226898193359375)


,Product_Category_1,Purchase
0,1,1910013754
4,5,941835229
7,8,854318799
5,6,324150302
1,2,268516186
2,3,204084713
15,16,145120612
10,11,113791115
9,10,100837301
14,15,92969042


In [31]:
sqlContext.registerDataFrameAsTable(df,'df')
start=time.time()
v11=sqlContext.sql('''SELECT Product_Category_1, sum(Purchase) as Purchase
                            FROM df 
                     GROUP BY Product_Category_1''')
v11.show()
end=time.time()

print(end-start)

+------------------+----------+
|Product_Category_1|  Purchase|
+------------------+----------+
|                12|   5331844|
|                 1|1910013754|
|                13|   4008601|
|                 6| 324150302|
|                16| 145120612|
|                 3| 204084713|
|                20|    944727|
|                 5| 941835229|
|                19|     59378|
|                15|  92969042|
|                17|   5878699|
|                 9|   6370324|
|                 4|  27380488|
|                 8| 854318799|
|                 7|  60896731|
|                10| 100837301|
|                11| 113791115|
|                14|  20014696|
|                 2| 268516186|
|                18|   9290201|
+------------------+----------+

0.415045022964


In [37]:
#groub by 12 
start=time.time()

prod=train.groupby(["City_Category",'Product_Category_1'])['Purchase'].sum().reset_index()

end=time.time()

print("time taken is",end-start)
prod

('time taken is', 0.037880897521972656)


,City_Category,Product_Category_1,Purchase
0,A,1,457708014
1,A,2,66958730
2,A,3,49590060
3,A,4,6922707
4,A,5,258240117
5,A,6,85467361
6,A,7,19909313
7,A,8,235809873
8,A,9,1630326
9,A,10,26034148


In [32]:
sqlContext.registerDataFrameAsTable(df,'df')
start=time.time()
v12=sqlContext.sql('''SELECT City_Category,Product_Category_1, sum(Purchase) as Purchase
                            FROM df 
                     GROUP BY City_Category,Gender,Product_Category_1
                     order by City_Category,Product_Category_1
                              ''')
v12.show()
end=time.time()

print(end-start)

+-------------+------------------+---------+
|City_Category|Product_Category_1| Purchase|
+-------------+------------------+---------+
|            A|                 1|376181738|
|            A|                 1| 81526276|
|            A|                 2| 16027983|
|            A|                 2| 50930747|
|            A|                 3| 35592296|
|            A|                 3| 13997764|
|            A|                 4|  4893956|
|            A|                 4|  2028751|
|            A|                 5|186780661|
|            A|                 5| 71459456|
|            A|                 6| 19444225|
|            A|                 6| 66023136|
|            A|                 7|  5376110|
|            A|                 7| 14533203|
|            A|                 8|167715960|
|            A|                 8| 68093913|
|            A|                 9|   232955|
|            A|                 9|  1397371|
|            A|                10| 20251232|
|         

In [35]:
##WRITE CODE HERE WITH NEW STYLE
start=time.time()

gen_1= df.groupby('Marital_Status')\
        .agg(sum('Purchase'))\
        .orderBy(sum('Purchase'),ascending=False)


gen_1.show()



end=time.time()

print(end-start)
  



In [ ]:
##WRITE CODE HERE WITH NEW STYLE
start=time.time()

gen_1= df.groupby('Gender')\
    .agg(sum('Purchase'))\
        .orderBy(sum('Purchase'),ascending=False)


gen_1.show()



end=time.time()

print(end-start)

In [125]:
df.show

<bound method DataFrame.show of DataFrame[User_ID: int, Product_ID: string, Gender: int, Age: int, Occupation: int, City_Category: string, Stay_In_Current_City_Years: int, Marital_Status: int, Product_Category_1: int, Product_Category_2: int, Product_Category_3: int, Purchase: int, City_Category1: int]>

In [27]:
cust_py_test

DataFrame[User_ID: int, Product_ID: string, Gender: int, Age: int, Occupation: int, City_Category: string, Stay_In_Current_City_Years: int, Marital_Status: int, Product_Category_1: int, Product_Category_2: int, Product_Category_3: int, City_Category1: int]

## Linear Regression

In [28]:
from pyspark.ml.feature import VectorAssembler
t= VectorAssembler(inputCols=['Gender','Marital_Status','Occupation','Age','Stay_In_Current_City_Years','City_Category1'], outputCol='Purchase2')
training=t.transform(cust_py)\
          .withColumn('Occupation', cust_py.Occupation)
training.toPandas().head()

test=t.transform(cust_py_test)\

In [29]:
test=t.transform(cust_py_test)\

In [57]:
test

DataFrame[User_ID: int, Product_ID: string, Gender: int, Age: int, Occupation: int, City_Category: string, Stay_In_Current_City_Years: int, Marital_Status: int, Product_Category_1: int, Product_Category_2: int, Product_Category_3: int, City_Category1: int, Purchase2: vector]

In [30]:
training.toPandas().shape
test.toPandas().shape

(233599, 13)

In [31]:
training_input=training.select('Gender','City_Category1','Marital_Status','Occupation','Age','Stay_In_Current_City_Years').rdd.map(lambda x: x[0]).collect
training_Purchase= training.select('Purchase2').rdd.map(lambda x: x[0]).collect




#all_yrs=training_input+prediction_yrs

In [32]:

test_input=training.select('Gender','City_Category1','Marital_Status','Occupation','Age','Stay_In_Current_City_Years').rdd.map(lambda x: x[0]).collect
test_Purchase= test.select('Purchase2').rdd.map(lambda x: x[0]).collect

In [33]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['Gender','City_Category1','Marital_Status','Occupation','Age','Stay_In_Current_City_Years'], outputCol = 'Purchase2')
cust_py_new = vectorAssembler.transform(cust_py)
cust_py_new_test = vectorAssembler.transform(cust_py_test)
#cust_py_new.show()


In [34]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

Coefficients: [696.6880980866673,-61.99935519753688,7.747668220310474,3.92069404898097,10.21058576575375,383.43217838478165]
Intercept: 7808.81355971


In [50]:
start=time.time()
lr = LinearRegression(featuresCol = 'Purchase2', labelCol='Purchase', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(training)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))
lr_predictions = lr_model.transform(cust_py_new_test)
lr_predictions.select("prediction",'Gender','Marital_Status','City_Category1','Occupation','Age','Stay_In_Current_City_Years',"Purchase2").show(5)
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="prediction",metricName="r2")
#print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

end=timeit.timeit()

end=time.time()

print(end-start)

Coefficients: [696.6880980866673,-61.99935519753688,7.747668220310474,3.92069404898097,10.21058576575375,383.43217838478165]
Intercept: 7808.81355971
+-----------------+------+--------------+--------------+----------+---+--------------------------+--------------------+
|       prediction|Gender|Marital_Status|City_Category1|Occupation|Age|Stay_In_Current_City_Years|           Purchase2|
+-----------------+------+--------------+--------------+----------+---+--------------------------+--------------------+
|9673.667947486518|     1|             1|             2|         7| 48|                         2|[1.0,2.0,1.0,7.0,...|
|8692.472964005348|     1|             0|             3|        17| 30|                         0|[1.0,3.0,0.0,17.0...|
|9638.635355749499|     0|             1|             2|         1| 40|                         4|[0.0,2.0,1.0,1.0,...|
|9638.635355749499|     0|             1|             2|         1| 40|                         4|[0.0,2.0,1.0,1.0,...|
|8316.4859

In [36]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 5045.123598
r2: 0.007343


In [41]:
print(end-start)

32.9003689289
